In [1]:
library(lme4)
library(lmerTest)
library(sjPlot)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




# Load data

In [2]:
df <- read.csv("output/critical_long_format.csv")
#df <- read.csv("output/critical_long_format_with_dummy.csv")
df$ID <- df$client_id

# Data preparation

In [3]:
# center binary variables
df$critical_cent <- scale(df$critical, scale=F)
df$mail_cent <- scale(df$mailed, scale=F)
df$mailday_cent <- scale(df$mailday, scale=F)
df$weekday_cent <- scale(df$is_weekday, scale=F)


In [4]:
# factors for green hours, mail and mailday
df$critical_f <- as.factor(df$critical)
levels(df$critical_f) <- c("No green hours","Green hours (11am to 3pm)")
df$mail_f <- as.factor(df$mailed)
levels(df$mail_f) <- c("No mail","Green mail")
df$mail_day_f <- as.factor(df$mailday)
levels(df$mail_day_f) <- c("No mail for day","Mail for day")
df$weekday_f <- as.factor(df$is_weekday)
levels(df$weekday_f) <- c("Mo-Fr","Sa-Su")

In [5]:
df_maildays = df[df$mailday == 1,]
df_nomaildays = df[df$mailday == 0,]

df_nomail = df[df$mailed == 0,]

# Analyzing start of chargers

## MailDays

### Model: `Charge ~ mail*critical + 1|D`

In [6]:
m1 <- lmer(charge ~ mail_cent*critical_cent+(1|ID),data=df_maildays)
summary(m1)
#tab_model(m1)


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * critical_cent + (1 | ID)
   Data: df_maildays

REML criterion at convergence: 29370

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3239 -0.2412  0.0067  0.0540 25.5930 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.179    0.4231  
 Residual             1.946    1.3948  
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)             7.034e-02  3.017e-02 4.094e+02   2.331   0.0202 *  
mail_cent               2.602e-01  3.152e-02 8.211e+03   8.255  < 2e-16 ***
critical_cent           3.073e-02  3.697e-02 7.947e+03   0.831   0.4060    
mail_cent:critical_cent 3.877e-01  6.138e-02 7.947e+03   6.316 2.83e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt crt

### Model: `Charge ~ mail*critical*weekday + 1|D`

In [7]:
m1 <- lmer(charge ~ mail_cent*weekday_cent*critical_cent+(1|ID),data=df_maildays)
summary(m1)
#tab_model(m1)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * weekday_cent * critical_cent + (1 | ID)
   Data: df_maildays

REML criterion at convergence: 29368

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4257 -0.1737  0.0104  0.0514 25.5103 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.1791   0.4232  
 Residual             1.9424   1.3937  
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                             0.06518    0.03076  441.52004   2.119
mail_cent                               0.24581    0.03294 8210.19974   7.461
weekday_cent                           -0.03421    0.03733 8050.95044  -0.917
critical_cent                           0.02898    0.03880 7942.96902   0.747
mail_cent:weekday_cent                 -0.10297    0.06320 8207.65265  -1.629
mail_cent:critical_cent 

## No Mail Days

### Model: `Charge ~ critical*weekday + 1|ID`


In [8]:
m2 <- lmer(charge ~ critical_cent*weekday_cent+(1|ID),data=df_nomaildays)
summary(m2)
#tab_model(m1)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ critical_cent * weekday_cent + (1 | ID)
   Data: df_nomaildays

REML criterion at convergence: 50890.6

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.717 -0.040 -0.001  0.001 47.331 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07152  0.2674  
 Residual             0.89621  0.9467  
Number of obs: 18444, groups:  ID, 318

Fixed effects:
                             Estimate Std. Error         df t value Pr(>|t|)
(Intercept)                 6.422e-02  1.657e-02  3.199e+02   3.875  0.00013
critical_cent              -3.280e-02  1.412e-02  1.812e+04  -2.323  0.02018
weekday_cent                2.769e-02  1.629e-02  1.812e+04   1.700  0.08917
critical_cent:weekday_cent -1.907e-02  3.258e-02  1.812e+04  -0.585  0.55836
                              
(Intercept)                ***
critical_cent              *  
weekday_cent               .  


## ALL: Mail and No Mail Days

### Model: `Charge ~ mail*critical*weekday  + mailday_cent + 1|ID`

In [9]:
m4 <- lmer(charge ~ mail_cent*critical_cent*weekday_cent + mailday_cent +(1|ID),data=df)
summary(m4)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * critical_cent * weekday_cent + mailday_cent +  
    (1 | ID)
   Data: df

REML criterion at convergence: 82349.9

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.868 -0.057 -0.003  0.024 40.299 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07607  0.2758  
 Residual             1.24847  1.1174  
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                           9.149e-02  1.693e-02  3.185e+02   5.404
mail_cent                             2.417e-01  2.549e-02  2.655e+04   9.483
critical_cent                         2.075e-02  1.378e-02  2.639e+04   1.506
weekday_cent                         -4.182e-03  1.522e-02  2.639e+04  -0.275
mailday_cent                         -3.198e-02  1.948e-02  2.646e+04  -1.641
mail_cent:critical_

### Model: `Charge ~ mailday*critical*weekday  + 1|ID`

In [10]:
m5 <- lmer(charge ~ mailday_cent*critical_cent*weekday_cent +(1|ID),data=df)
summary(m5)
#tab_model(m4)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mailday_cent * critical_cent * weekday_cent + (1 | ID)
   Data: df

REML criterion at convergence: 82531.8

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.000 -0.094 -0.012  0.023 40.168 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07513  0.2741  
 Residual             1.25740  1.1213  
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                          Estimate Std. Error         df
(Intercept)                              9.018e-02  1.690e-02  3.219e+02
mailday_cent                             8.387e-02  1.537e-02  2.639e+04
critical_cent                            2.215e-02  1.404e-02  2.639e+04
weekday_cent                            -7.197e-04  1.537e-02  2.639e+04
mailday_cent:critical_cent               1.775e-01  3.073e-02  2.639e+04
mailday_cent:weekday_cent               -9.179e-02  3.137e-02  2.63

### Model: `Charge ~ mailday*critical*weekday  + mail + 1|ID`

In [11]:
m6 <- lmer(charge ~ mail_cent + mailday_cent*critical_cent*weekday_cent + (1|ID),data=df)
summary(m6)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent + mailday_cent * critical_cent * weekday_cent +  
    (1 | ID)
   Data: df

REML criterion at convergence: 82425.5

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.966 -0.115 -0.009  0.025 40.241 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07603  0.2757  
 Residual             1.25199  1.1189  
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                          Estimate Std. Error         df
(Intercept)                              8.996e-02  1.697e-02  3.218e+02
mail_cent                                2.632e-01  2.486e-02  2.656e+04
mailday_cent                            -4.499e-02  1.958e-02  2.646e+04
critical_cent                            2.215e-02  1.401e-02  2.639e+04
weekday_cent                            -2.153e-03  1.533e-02  2.639e+04
mailday_cent:critical_cent               1.775e-0

### Model: `Charge ~ mail*mailday*critical*weekday  + 1|ID`

In [12]:
m7 <- lmer(charge ~ mail_cent*mailday_cent*critical_cent*weekday_cent + (1|ID),data=df)
summary(m7)

fixed-effect model matrix is rank deficient so dropping 4 columns / coefficients



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * mailday_cent * critical_cent * weekday_cent +  
    (1 | ID)
   Data: df

REML criterion at convergence: 82360.7

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.868 -0.057 -0.005  0.026 40.295 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07607  0.2758  
 Residual             1.24853  1.1174  
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                          Estimate Std. Error         df
(Intercept)                              9.010e-02  1.697e-02  3.218e+02
mail_cent                                2.474e-01  2.595e-02  2.656e+04
mailday_cent                            -3.687e-02  1.993e-02  2.646e+04
critical_cent                            2.221e-02  1.399e-02  2.638e+04
weekday_cent                            -2.197e-03  1.531e-02  2.638e+04
mail_cent:critical_cent                  3.449e-0

In [13]:
m71 <- lmer(charge ~ mail_cent+mailday_cent+critical_cent+weekday_cent + (1|ID),data=df)
summary(m71)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent + mailday_cent + critical_cent + weekday_cent +  
    (1 | ID)
   Data: df

REML criterion at convergence: 82464.2

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.018 -0.153  0.000  0.024 40.237 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.07599  0.2757  
 Residual             1.25455  1.1201  
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    9.450e-02  1.691e-02  3.170e+02   5.589 4.94e-08 ***
mail_cent      2.621e-01  2.489e-02  2.656e+04  10.533  < 2e-16 ***
mailday_cent  -3.784e-02  1.946e-02  2.646e+04  -1.944   0.0519 .  
critical_cent  2.631e-02  1.371e-02  2.639e+04   1.920   0.0549 .  
weekday_cent  -8.773e-03  1.520e-02  2.639e+04  -0.577   0.5638    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation

## No mail

### Model: `Charge ~ mailday*critical*weekday + 1|ID`

In [14]:
m8 <- lmer(charge ~ mailday_cent*critical_cent*weekday_cent + (1|ID),data=df_nomail)
summary(m8)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mailday_cent * critical_cent * weekday_cent + (1 | ID)
   Data: df_nomail

REML criterion at convergence: 60363.2

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.471 -0.047 -0.006 -0.003 49.701 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.05667  0.2381  
 Residual             0.81567  0.9031  
Number of obs: 22686, groups:  ID, 318

Fixed effects:
                                          Estimate Std. Error         df
(Intercept)                              5.326e-02  1.485e-02  3.451e+02
mailday_cent                            -3.542e-02  1.612e-02  2.245e+04
critical_cent                           -2.977e-02  1.295e-02  2.237e+04
weekday_cent                             1.369e-02  1.378e-02  2.241e+04
mailday_cent:critical_cent               9.796e-03  3.209e-02  2.237e+04
mailday_cent:weekday_cent               -4.523e-02  3.197e-0

### Model: `Charge ~ mailday+critical+weekday + 1|ID`

In [15]:
m9 <- lmer(charge ~ mailday_cent+critical_cent+weekday_cent + (1|ID),data=df_nomail)
summary(m9)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mailday_cent + critical_cent + weekday_cent + (1 | ID)
   Data: df_nomail

REML criterion at convergence: 60346.7

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.465 -0.041 -0.009 -0.004 49.708 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.05668  0.2381  
 Residual             0.81562  0.9031  
Number of obs: 22686, groups:  ID, 318

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    5.548e-02  1.476e-02  3.374e+02   3.758 0.000202 ***
mailday_cent  -3.059e-02  1.576e-02  2.245e+04  -1.941 0.052227 .  
critical_cent -3.290e-02  1.199e-02  2.237e+04  -2.744 0.006082 ** 
weekday_cent   1.700e-02  1.358e-02  2.240e+04   1.252 0.210616    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) mldy_c crtcl_
mailday_cnt 0.128          